In [ ]:
#| default_exp inspecttools

# inspecttools
> Solveit tools for inspecting the symbol table and importing modules

This module provides **LLM tools** for Solveit to dynamically inspect source code, types, and module capabilities. Functions take *string* arguments (dotted symbol paths) rather than Python objects because LLM tool interfaces can only pass serializable values—not live Python references.

In [ ]:
#| export
import inspect, re
from importlib import import_module
from dialoghelper import add_msg

In [ ]:
#| export
def _find_frame_dict(var:str):
    "Find the dict (globals or locals) containing var"
    frame = inspect.currentframe().f_back
    while frame:
        if var in frame.f_globals: return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [ ]:
#| export
def doimport(mod: str):
    "Import a module into the caller's global namespace"
    g = _find_frame_dict('__msg_id')
    import_module(mod)
    g[mod.split('.')[0]] = import_module(mod.split('.')[0])

`doimport` lets the LLM dynamically import modules by name. Here we import `fastcore.utils` and verify it's available.

In [ ]:
doimport('fastcore.utils')
fastcore.__version__

'1.9.3'

In [ ]:
#| export
_last = None

def resolve(
    sym: str  # Dotted symbol path, with optional [n] indexing, e.g. "module.attr.subattr[1]" or "_last" for previous result
):
    """Resolve a dotted symbol string to its Python object, with optional [n] indexing. Sets global `_last` to the resolved object for chaining. Pass `"_last"` to reference the result of the previous tool call.

    Examples:

    - `resolve("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `resolve("mylist[2]")` -> third element of mylist"""
    global _last
    if sym == '_last': return _last
    parts = re.split(r'\.(?![^\[]*\])', sym)
    g = _find_frame_dict('__msg_id')
    obj = _last if parts[0] == '_last' else g[parts[0]]
    for part in parts[1:]:
        match = re.match(r'(\w+)\[(\d+)\]$', part)
        if match:
            attr, idx = match.groups()
            obj = getattr(obj, attr)
            obj = obj[int(idx)]
        else: obj = getattr(obj, part)
    _last = obj
    return obj

`resolve` navigates dotted paths like `"a.argfirst"` to reach the actual Python object.

In [ ]:
a = fastcore.utils.L(1)
resolve('a.argfirst')

<bound method L.argfirst of [1]>

It also sets `_last` for chaining. Since it's used internally by all `get*` tools in this module, the tools all set `_last` too.

In [ ]:
_last

<bound method L.argfirst of [1]>

It works on both objects and classes:

In [ ]:
resolve('fastcore.utils.L.argfirst')

<function fastcore.foundation.L.argfirst(self: fastcore.foundation.L, f, negate=False)>

In [ ]:
#| export
def symsrc(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    """Get the source code for a symbol.

    Examples:

    - `symsrc("sympy.sets.sets.Interval")` -> source code of Interval class
    - `symsrc("_last")` -> source of object from previous tool call
    - For dispatchers or registries of callables: `getnth("module.dispatcher.funcs", n) then symsrc("_last")`"""
    obj = resolve(sym)
    source = inspect.getsource(obj)
    filename = inspect.getfile(obj)
    return f"File: {filename}\n\n{source}"

`symsrc` retrieves the source code of any symbol by path—essential for letting the LLM understand how functions work.

In [ ]:
print(symsrc('a.argfirst'))

File: /Users/jhoward/aai-ws/fastcore/fastcore/foundation.py

@patch
def argfirst(self:L, f, negate=False):
    "Return index of first matching item"
    if negate: f = not_(f)
    return first(i for i,o in self.enumerate() if f(o))



In [ ]:
#| export
def showsrc(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    "Create a note to show the user (and following LLM prompts) the source of `sym`, following `symsrc` rules"
    res = symsrc(sym)
    add_msg(f'```python\n{res}\n```')
    return 'Message has been added to dialog successfully'

`showsrc` is like `symsrc` but adds the information as a note message.

In [ ]:
#| export
def gettype(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    """Get the type of a symbol and set `_last`.

    Examples:

    - `gettype("sympy.sets.sets.Interval")` -> `<class 'type'>`
    - `gettype("_last")` -> type of previous result"""
    return type(resolve(sym))

`gettype` returns the type of a symbol—useful for the LLM to understand what kind of object it's dealing with.

In [ ]:
gettype('a.argfirst')

method

In [ ]:
#| export
def getdir(
    sym: str,  # Dotted symbol path or "_last" for previous result
    exclude_private: bool=False # Filter out attrs starting with "_"
):
    """Get dir() listing of a symbol's attributes and set `_last`. E.g: `getdir("sympy.Interval")` -> `['__add__', '__and__', ...]`"""
    res = dir(resolve(sym))
    if not exclude_private: return res
    return [o for o in res if o[0]!='_']

`getdir` lists all attributes of an object (i.e it calls `dir()`. Here we filter out private names to see the public API of an `L` list.

In [ ]:
' '.join(getdir('a', exclude_private=True))

'accumulate append argfirst argwhere attrgot batched clear combinations compress concat copy copy count cycle dropwhile enumerate extend filter flatten groupby index insert itemgot items map map_dict map_first map_zip map_zipwith pairwise partition permutations pop product range reduce remove renumerate reverse rstarargfirst rstarargwhere rstardropwhile rstarfilter rstarmap rstarpartition rstarreduce rstarsorted rstartakewhile setattrs shuffle sort sorted split splitlines starargfirst starargwhere stardropwhile starfilter starmap starpartition starreduce starsorted startakewhile sum takewhile unique val2idx zip zipwith'

In [ ]:
#| export
def getval(
    sym: str  # Dotted symbol path or "_last" for previous result
):
    """Get repr of a symbol's value and set `_last`.

    Examples:
    
    - `getval("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `getval("some_dict.keys")` -> `dict_keys([...])`"""
    return repr(resolve(sym))

`getval` returns the `repr()` of a symbol's value—handy for inspecting data without needing to execute arbitrary code.

In [ ]:
getval('a')

'[1]'

In [ ]:
#| export
def getnth(
    sym: str,  # Dotted symbol path to a dict or object with .values()
    n: int     # Index into the values (0-based)
):
    """Get the nth value from a dict (or any object with .values()). Sets `_last` so you can chain with `symsrc("_last")` etc.

    Examples:
    
    - `getnth("dispatcher.funcs", 12)` -> 13th registered function
    - `getnth("dispatcher.funcs", 0); symsrc("_last")` -> source of first handler"""
    global _last
    _last = list(resolve(sym).values())[n]
    return _last

`getnth` extracts the nth value from a dict (or anything with `.values()`).

In [ ]:
handlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))
getnth('handlers', 0)

<function __main__.<lambda>(x)>

Combined with `_last`, this lets the LLM drill into registries of handlers/dispatchers and then inspect their source.

In [ ]:
symsrc('_last')

'File: /var/folders/51/b2_szf2945n072c0vj2cyty40000gn/T/ipykernel_66207/2745627804.py\n\nhandlers = dict(int=lambda x: x*2, str=lambda x: x.upper(), list=lambda x: len(x))\n'

In [ ]:
#| export
def run_code_interactive(
    code:str # Code to have user run
):
    """Insert code into user's dialog and request for the user to run it. Use other tools where possible, 
    but if they can not find needed information, *ALWAYS* use this instead of guessing or giving up.
    IMPORTANT: This tool is TERMINAL - after calling it, you MUST stop all tool usage 
    and wait for user response. Never call additional tools after this one."""
    add_msg('# Please run this:\n'+code, msg_type='code')
    return "CRITICAL: Message added to user dialog. STOP IMMEDIATELY. Do NOT call any more tools. Wait for user to run code and respond."

In [ ]:
#| export
def inspect_tool_info():
    add_msg('Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,run_code_interactive]`')

In [ ]:
# inspect_tool_info()

Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,run_code_interactive]`